In [11]:
import time

import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)

import sys
sys.path.append("../tfti")
from tfti_batched_inference import *

In [2]:
config = get_config(
    problem="genomics_binding_deepsea_gm12878",
    model="tfti_transformer",
    hparams_set="tfti_transformer_base",
    hparams="",
    checkpoint_path="~/Downloads/6-64-25/model.ckpt-210001",
)

preprocess_batch_fn = get_preprocess_batch_fn(config)
inference_fn = get_inference_fn(config)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


[2018-05-02 15:41:54,709] From /Users/alexyku/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensor2tensor/layers/common_layers.py:553: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
line = """NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAATTTGAATTTGAATTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAAATTTGAAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTAATTTTAATTTTAATTTTAATTTTAATTTTAATTTGAATTTTGAATTTGAATTTGAATTTGATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAATTTGAATTTGTCTCCGATTTGTACTTCCTTTCTTTCTTTCCCGTGCATTGCACCACAATGCGCTGTTCTTGGTACACGATTATTCAAAGTGCGCTACACCATAATCTACTGTTCTTTGTCTCCGCTGTGTTCCCGTGCGCTG	0000000000000000000000000000000000000000000000000000000000001000000100000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000010001101000000000000000000000010000000000000000000000000000000000000000000000000000000"""
tab_split = line.split("\t")

# string inputs
inputs_str = tab_split[0].strip()
targets_str = tab_split[1].strip()
keep_mask_str = "0" * len(targets_str)

# numpy array inputs
inputs_np = np.array(list(map(ord, inputs_str)))
targets_np = np.array(list(map(int, targets_str)))
keep_mask_np = np.array(list(map(int, keep_mask_str))).astype(bool)

def pseudo_batch(x, n):
    """Yields the value x n-times."""
    for _ in range(n):
        yield x

In [7]:
# string inputs
example_batch = preprocess_batch_fn(
    pseudo_batch(inputs_str, 16),
    pseudo_batch(targets_str, 16),
    pseudo_batch(keep_mask_str, 16)
)
fetch_batch = inference_fn(example_batch)

for k, v in example_batch.items():
    print(f"{(k+':').ljust(20)} dtype={str(v.dtype).ljust(10)} shape={v.shape}")
for k, v in fetch_batch.items():
    print(f"{(k+':').ljust(20)} dtype={str(v.dtype).ljust(10)} shape={v.shape}")

inputs:              dtype=int64      shape=(16, 250, 1, 1)
targets:             dtype=int32      shape=(16, 35, 1, 1)
latents:             dtype=int32      shape=(16, 35, 1, 1)
metrics_weights:     dtype=float32    shape=(16, 35, 1, 1)
labels:              dtype=int32      shape=(16, 35, 1, 1)
logits:              dtype=float32    shape=(16, 35, 1, 1)
predictions:         dtype=float32    shape=(16, 35, 1, 1)


In [8]:
# numpy array inputs
example_batch = preprocess_batch_fn(
    pseudo_batch(inputs_np, 16),
    pseudo_batch(targets_np, 16),
    pseudo_batch(keep_mask_np, 16)
)
fetch_batch = inference_fn(example_batch)

for k, v in example_batch.items():
    print(f"{(k+':').ljust(20)} dtype={str(v.dtype).ljust(10)} shape={v.shape}")
for k, v in fetch_batch.items():
    print(f"{(k+':').ljust(20)} dtype={str(v.dtype).ljust(10)} shape={v.shape}")

inputs:              dtype=int64      shape=(16, 250, 1, 1)
targets:             dtype=int32      shape=(16, 35, 1, 1)
latents:             dtype=int32      shape=(16, 35, 1, 1)
metrics_weights:     dtype=float32    shape=(16, 35, 1, 1)
labels:              dtype=int32      shape=(16, 35, 1, 1)
logits:              dtype=float32    shape=(16, 35, 1, 1)
predictions:         dtype=float32    shape=(16, 35, 1, 1)
